In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config('spark.shuffle.useOldFetchProtocol', 'true'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

#### Loading loan defaulters data

In [2]:
loan_defaulters_data_df = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.load("/user/itv018355/lendingclubproject/raw/loan_defaulters_data_csv")

In [3]:
loan_defaulters_data_df

+--------------------+-----------+-----------+-------+--------------------+--------------+------------------+----------------------+----------------------+
|           member_id|delinq_2yrs|delinq_amnt|pub_rec|pub_rec_bankruptcies|inq_last_6mths|total_rec_late_fee|mths_since_last_delinq|mths_since_last_record|
+--------------------+-----------+-----------+-------+--------------------+--------------+------------------+----------------------+----------------------+
|a4ec00ba67fadf2fe...|        1.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                   9.0|                  null|
|4f7a9e6ffaacd5da2...|        1.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  14.0|                  null|
|e935a4c27fc78ae61...|        0.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  43.0|                  null|
|2d32004bd5e1dc3c3...|        1.0|        0.0|    1.0|                 0.0|           2.0|               0.0|                  17.0|                  58.0|
|f7116b7f7546a7952...|        0.0|        0.0|    2.0|                 1.0|           0.0|               0.0|                  28.0|                  55.0|
|d3aa3a3c95eca5631...|        0.0|        0.0|    0.0|                 0.0|           0.0|              68.8|                  null|                  null|
|fc8a2e046eaaba02d...|        0.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  null|                  null|
|577ae670ac2ec7ed3...|        0.0|        0.0|    0.0|                 0.0|           1.0|               0.0|                  null|                  null|
|d3792868819649ba9...|        0.0|        0.0|    0.0|                 0.0|           0.0|            110.13|                  null|                  null|
|6d3a5a422261348b3...|        0.0|        0.0|    1.0|                 0.0|           1.0|               0.0|                  null|                  38.0|
|7435a5a4b2b825a20...|        0.0|        0.0|    0.0|                 0.0|           3.0|               0.0|                  null|                  null|
|2a1dd1ae5b20c4dee...|        0.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  69.0|                  null|
|8d203efd7c1af010c...|        0.0|        0.0|    1.0|                 1.0|           0.0|               0.0|                  null|                  75.0|
|c266c3c131861bb8c...|        0.0|        0.0|    1.0|                 0.0|           0.0|               0.0|                  24.0|                  81.0|
|c7b46f35c566b6d56...|        0.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  null|                  null|
|9284503d1d6dcd23b...|        0.0|        0.0|    1.0|                 1.0|           0.0|               0.0|                  null|                  74.0|
|616e44da25c28bd55...|        0.0|        0.0|    0.0|                 0.0|           0.0|            134.19|                  null|                  null|
|8192d3261d35b28af...|        0.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  null|                  null|
|c764f6b4666a3a439...|        0.0|        0.0|    0.0|                 0.0|           1.0|               0.0|                  null|                  null|
|199119e026b4bfb14...|        1.0|        0.0|    2.0|                 0.0|           0.0|               0.0|                  21.0|                  44.0|
+--------------------+-----------+-----------+-------+--------------------+--------------+------------------+----------------------+----------------------+
only showing top 20 rows

In [4]:
loan_defaulters_data_df.createOrReplaceTempView("loan_defaulters_data")

#### let's check the different values in delinq_2yrs column

In [5]:
spark.sql("select distinct(delinq_2yrs)from loan_defaulters_data")

+--------------------+
|         delinq_2yrs|
+--------------------+
|                 1.0|
|I bike to work on...|
|271 monthly payme...|
|VISA and AMEX cre...|
|etc. and I feel t...|
|               183xx|
|                  AZ|
|               017xx|
|               923xx|
|               446xx|
|who is a financia...|
|and all of that c...|
|                  LA|
|               24.66|
|and the removal o...|
|I would like to c...|
|               12.24|
|                22.0|
|so most of my inc...|
|I would like to p...|
+--------------------+
only showing top 20 rows

In [6]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters_data group by delinq_2yrs order by total desc").show(40)

+------------------+-------+
|       delinq_2yrs|  total|
+------------------+-------+
|               0.0|1838878|
|               1.0| 281335|
|               2.0|  81285|
|               3.0|  29539|
|               4.0|  13179|
|               5.0|   6599|
|               6.0|   3717|
|               7.0|   2062|
|               8.0|   1223|
|               9.0|    818|
|              10.0|    556|
|              11.0|    363|
|              12.0|    264|
|              13.0|    165|
|              14.0|    120|
|              15.0|     87|
|              null|     63|
|              16.0|     55|
|              18.0|     30|
|              17.0|     30|
|              19.0|     23|
|              20.0|     17|
|              21.0|     12|
|                CA|      8|
|                TX|      6|
|                IL|      5|
|debt_consolidation|      5|
|              22.0|      5|
|    small_business|      5|
|                FL|      4|
|              24.0|      4|
|             

#### Defining schema

In [23]:
loan_defaulters_schema = "member_id string, delinq_2yrs float, delinq_amnt float, pub_rec float, pub_rec_bankruptcies float, inq_last_6mths float, total_rec_late_fee float, mths_since_last_delinq float, mths_since_last_record float"

In [24]:
loan_defaulters_data_df = spark.read \
.format("csv") \
.option("header",True) \
.schema(loan_defaulters_schema) \
.load("/user/itv018355/lendingclubproject/raw/loan_defaulters_data_csv")

In [25]:
loan_defaulters_data_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- delinq_2yrs: float (nullable = true)
 |-- delinq_amnt: float (nullable = true)
 |-- pub_rec: float (nullable = true)
 |-- pub_rec_bankruptcies: float (nullable = true)
 |-- inq_last_6mths: float (nullable = true)
 |-- total_rec_late_fee: float (nullable = true)
 |-- mths_since_last_delinq: float (nullable = true)
 |-- mths_since_last_record: float (nullable = true)



In [26]:
loan_defaulters_data_df.createOrReplaceTempView("loan_defaulters_data")

In [27]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters_data group by delinq_2yrs order by total desc").show(40)

+-----------+-------+
|delinq_2yrs|  total|
+-----------+-------+
|        0.0|1838878|
|        1.0| 281335|
|        2.0|  81285|
|        3.0|  29539|
|        4.0|  13179|
|        5.0|   6599|
|        6.0|   3717|
|        7.0|   2062|
|        8.0|   1223|
|        9.0|    818|
|       10.0|    556|
|       11.0|    363|
|       12.0|    264|
|       null|    261|
|       13.0|    165|
|       14.0|    120|
|       15.0|     87|
|       16.0|     55|
|       17.0|     30|
|       18.0|     30|
|       19.0|     23|
|       20.0|     17|
|       21.0|     12|
|       22.0|      5|
|       24.0|      4|
|       26.0|      3|
|       25.0|      2|
|       23.0|      2|
|       30.0|      2|
|       29.0|      2|
|       3.44|      2|
|       27.0|      1|
|      13.76|      1|
|      26.24|      1|
|      22.62|      1|
|       1.37|      1|
|       39.0|      1|
|       3.45|      1|
|       5.52|      1|
|       14.1|      1|
+-----------+-------+
only showing top 40 rows



In [28]:
from pyspark.sql.functions import col

In [29]:
loan_defaulters_proc = loan_defaulters_data_df.withColumn("delinq_2yrs", col("delinq_2yrs").cast("integer")).fillna(0, subset = ["delinq_2yrs"])

In [30]:
loan_defaulters_proc.show(40)

+--------------------+-----------+-----------+-------+--------------------+--------------+------------------+----------------------+----------------------+
|           member_id|delinq_2yrs|delinq_amnt|pub_rec|pub_rec_bankruptcies|inq_last_6mths|total_rec_late_fee|mths_since_last_delinq|mths_since_last_record|
+--------------------+-----------+-----------+-------+--------------------+--------------+------------------+----------------------+----------------------+
|a4ec00ba67fadf2fe...|          1|        0.0|    0.0|                 0.0|           0.0|               0.0|                   9.0|                  null|
|4f7a9e6ffaacd5da2...|          1|        0.0|    0.0|                 0.0|           0.0|               0.0|                  14.0|                  null|
|e935a4c27fc78ae61...|          0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  43.0|                  null|
|2d32004bd5e1dc3c3...|          1|        0.0|    1.0|          

In [31]:
loan_defaulters_proc.createOrReplaceTempView("loan_defaulters_data")

In [32]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters_data group by delinq_2yrs order by total desc").show(40)

+-----------+-------+
|delinq_2yrs|  total|
+-----------+-------+
|          0|1839141|
|          1| 281337|
|          2|  81285|
|          3|  29545|
|          4|  13180|
|          5|   6601|
|          6|   3719|
|          7|   2063|
|          8|   1226|
|          9|    821|
|         10|    558|
|         11|    363|
|         12|    266|
|         13|    167|
|         14|    123|
|         15|     90|
|         16|     56|
|         17|     33|
|         18|     32|
|         19|     24|
|         20|     19|
|         21|     16|
|         22|      7|
|         24|      6|
|         23|      5|
|         26|      4|
|         25|      2|
|         30|      2|
|         29|      2|
|         28|      1|
|         35|      1|
|         27|      1|
|         42|      1|
|         39|      1|
|         32|      1|
|         36|      1|
|         58|      1|
+-----------+-------+



In [33]:
loan_defaulters_proc

+--------------------+-----------+-----------+-------+--------------------+--------------+------------------+----------------------+----------------------+
|           member_id|delinq_2yrs|delinq_amnt|pub_rec|pub_rec_bankruptcies|inq_last_6mths|total_rec_late_fee|mths_since_last_delinq|mths_since_last_record|
+--------------------+-----------+-----------+-------+--------------------+--------------+------------------+----------------------+----------------------+
|a4ec00ba67fadf2fe...|          1|        0.0|    0.0|                 0.0|           0.0|               0.0|                   9.0|                  null|
|4f7a9e6ffaacd5da2...|          1|        0.0|    0.0|                 0.0|           0.0|               0.0|                  14.0|                  null|
|e935a4c27fc78ae61...|          0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  43.0|                  null|
|2d32004bd5e1dc3c3...|          1|        0.0|    1.0|                 0.0|           2.0|               0.0|                  17.0|                  58.0|
|f7116b7f7546a7952...|          0|        0.0|    2.0|                 1.0|           0.0|               0.0|                  28.0|                  55.0|
|d3aa3a3c95eca5631...|          0|        0.0|    0.0|                 0.0|           0.0|              68.8|                  null|                  null|
|fc8a2e046eaaba02d...|          0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  null|                  null|
|577ae670ac2ec7ed3...|          0|        0.0|    0.0|                 0.0|           1.0|               0.0|                  null|                  null|
|d3792868819649ba9...|          0|        0.0|    0.0|                 0.0|           0.0|            110.13|                  null|                  null|
|6d3a5a422261348b3...|          0|        0.0|    1.0|                 0.0|           1.0|               0.0|                  null|                  38.0|
|7435a5a4b2b825a20...|          0|        0.0|    0.0|                 0.0|           3.0|               0.0|                  null|                  null|
|2a1dd1ae5b20c4dee...|          0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  69.0|                  null|
|8d203efd7c1af010c...|          0|        0.0|    1.0|                 1.0|           0.0|               0.0|                  null|                  75.0|
|c266c3c131861bb8c...|          0|        0.0|    1.0|                 0.0|           0.0|               0.0|                  24.0|                  81.0|
|c7b46f35c566b6d56...|          0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  null|                  null|
|9284503d1d6dcd23b...|          0|        0.0|    1.0|                 1.0|           0.0|               0.0|                  null|                  74.0|
|616e44da25c28bd55...|          0|        0.0|    0.0|                 0.0|           0.0|            134.19|                  null|                  null|
|8192d3261d35b28af...|          0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  null|                  null|
|c764f6b4666a3a439...|          0|        0.0|    0.0|                 0.0|           1.0|               0.0|                  null|                  null|
|199119e026b4bfb14...|          1|        0.0|    2.0|                 0.0|           0.0|               0.0|                  21.0|                  44.0|
+--------------------+-----------+-----------+-------+--------------------+--------------+------------------+----------------------+----------------------+
only showing top 20 rows

In [34]:
loan_def_delinq_df = spark.sql("select member_id,delinq_2yrs, delinq_amnt, int(mths_since_last_delinq) from loan_defaulters_data where delinq_2yrs > 0 or mths_since_last_delinq > 0")

In [35]:
loan_def_delinq_df

+--------------------+-----------+-----------+----------------------+
|           member_id|delinq_2yrs|delinq_amnt|mths_since_last_delinq|
+--------------------+-----------+-----------+----------------------+
|a4ec00ba67fadf2fe...|          1|        0.0|                     9|
|4f7a9e6ffaacd5da2...|          1|        0.0|                    14|
|e935a4c27fc78ae61...|          0|        0.0|                    43|
|2d32004bd5e1dc3c3...|          1|        0.0|                    17|
|f7116b7f7546a7952...|          0|        0.0|                    28|
|2a1dd1ae5b20c4dee...|          0|        0.0|                    69|
|c266c3c131861bb8c...|          0|        0.0|                    24|
|199119e026b4bfb14...|          1|        0.0|                    21|
|0a9cef2d26384e853...|          1|        0.0|                    15|
|39ff608d7284a71f5...|          1|        0.0|                    21|
|ab5995646618ca3ac...|          1|        0.0|                     7|
|1689ffd13ffd85359...|          0|        0.0|                    67|
|5c073be0c3bc3a88b...|          0|        0.0|                    58|
|89887226f9b253d34...|          1|        0.0|                     6|
|e09d0bab88142c5c5...|          1|        0.0|                    20|
|00391bd517649336b...|          0|        0.0|                    51|
|a31b06f4bf9a9881b...|          0|        0.0|                    24|
|3fd949383320d1bd8...|          0|        0.0|                    42|
|beb4a5787b4fd8cff...|          0|        0.0|                    49|
|f16ae4c8190196b94...|          0|        0.0|                    55|
+--------------------+-----------+-----------+----------------------+
only showing top 20 rows

In [36]:
loan_def_records_inq_df = spark.sql("select member_id from loan_defaulters_data where pub_rec > 0.0 or pub_rec_bankruptcies > 0.0 or inq_last_6mths > 0.0")

In [37]:
loan_def_records_inq_df.createOrReplaceTempView("loan_defaulters_data")

In [38]:
loan_def_records_inq_df.count()

1070125

In [39]:
loan_def_delinq_df.write \
.format("csv") \
.option("header",True) \
.mode("overwrite") \
.option("path", "/user/itv018355/lendingclubproject/cleaned/loan_defaulters_delinq_csv") \
.save()

In [40]:
loan_def_delinq_df.write \
.format("parquet") \
.option("header",True) \
.mode("overwrite") \
.option("path", "/user/itv018355/lendingclubproject/cleaned/loan_defaulters_delinq_parquet") \
.save()

In [42]:
loan_def_records_inq_df.write \
.format("csv") \
.option("header",True) \
.mode("overwrite") \
.option("path", "/user/itv018355/lendingclubproject/cleaned/loan_defaulters_records_inq_csv") \
.save()

In [43]:
loan_def_records_inq_df.write \
.format("parquet") \
.option("header",True) \
.mode("overwrite") \
.option("path", "/user/itv018355/lendingclubproject/cleaned/loan_defaulters_records_inq_parquet") \
.save()